# RL wcst Agent 001

The first, simplest, RL agent. Showcasing the basic structure.

## Optimality

The optimal learning behaviour would be too simply choose the last correct action. This would equate to an update equation that totally saturates previous information & essentially only considers the most recent but of information.

In [243]:
# !conda activate dynocog
# !conda init
# !conda install pandas -y
# !conda install -c pytorch pytorch -y
# !conda install -c conda-forge numpyro
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import torch
import sys

In [244]:
# ---- Hyperparameters ----x
rules   = ['shape', 'number', 'color']
colors  = ['yellow', 'red', 'green', 'blue']
shapes  = ['star', 'triangle', 'circle', 'cross']
numbers = [1,2,3,4]
cards   = ['card 1', 'card 2', 'card 3', 'card 4']
matching_cards = {
  'card 1': {'color': 'red', 'shape':'circle', 'number':1},
  'card 2': {'color': 'green', 'shape':'triangle', 'number':2},
  'card 3': {'color': 'blue', 'shape':'cross', 'number':3}, 
  'card 4': {'color': 'yellow', 'shape':'star', 'number':4}}

# Build RL Environment

Now we build the RL environment that can be used to train the parameters.

In our model the RL environment reflects WCST.


## Data Structure

Dictionaries are far faster than lists in Python, as such we structure each sample as a dictionary. Each environmental sample has two components:
- Current card: the card drawn
- Target card: the correct matching option
- Rule: the matching rule

Each of these are captured in a dictionary & each observation put into a list object called 'item'. Each item is then added to a list (giving us a list of observations) which is later conververted into a enumerable object.

Return: a list of observations & outputs representing the RL environment.

In [245]:
f = open("../data/wcst.txt", "r")

all_items = []
for l in f.readlines():
    
    # ---- target card ----x
    ind = 'card ' + l[18]

    # ---- current card ----x
    curr_card = l[0:16].strip()
    curr_card = re.split('([0-9]+)', curr_card)
    curr_card = {'color': curr_card[2], 'shape':curr_card[0], 'number':curr_card[1]}

    # ---- matching rule + target card ----x
    rule = (l[27:27+8]).strip().replace('\"','')    
    target_card = matching_cards[ind]

    # ----- list current + target cards ----x
    item = {'curr_card': curr_card, 'target_card':target_card, 'rule':rule}
    all_items.append(item)

# RL Environment

Now we compile these observations in a class to create all the functionality we require.

## Expeience Replay (Replay Memory)
- **sample_environment**: We sample random pairings to decouple bias, this is known as experience replay in the literature.
- **next**: Sample the next $n$ observation in order.


In [328]:
class WCST_Environment:
    def __init__(self, all_items):
        self.all_items = all_items
        self.index     = 0
        self.n         = len(all_items)
        self.sample_idx = []
        self.index = 0
    
    def sample_environment(self, n_samples):
        idx = np.random.choice(range(0,self.n), n_samples)
        self.sample_idx = self.sample_idx.append(idx)
        return([self.all_items[i] for i in idx])
    
    def batch(self, n):
        """Return: n samples
           Note:
                - The function is cylical & will return the same sequence of samples periodically if a large batch is required.
                - If one wishes to 'reset' & sample from the start either:
                    1. set self.index == 0 & sample
                    2. sample from self.all_items directly
        """
        deck = []
        nt = self.index + n
        if nt > 100:    
            loops = nt // 100 - 1    # integer devision == devision drop remainder
                                    # number of full training loops
            res   = nt % 100         # modulo == devision return remainder only
                                    # number of items above 0
            # --- until end ---x
            for i in all_items[nt:]: deck.append(i)
            # --- add 'loops' full data loops ----x
            for i in range(loops):
                for a in all_items: deck.append(a)
            # --- remaining data ---x
            for i in all_items[:res]: deck.append(i)
            self.index = res

        else: 
            deck = all_items[self.index:nt]
            self.index = nt
        return(deck)

In [332]:
wcst = WCST_Environment(all_items)
batch = wcst.batch(7550)
wcst.index

50

7450

In [275]:
len(deck)

12

# Defining Computation

RL Class should contain:
- Environment
- Model (given trained parameters)
- Training Loop
- Trainable Parameters
- Graphics (training processes etc)

There are two types of trainable model parameters:
- Parameter values: torch.tensor()
- Parameter distributions: numpy object()


# Model

We are working with a bandit as we only have $1$ state. We wish to maximise return by selecting the action that yields the highest expected return, that is, our policy $\pi(\alpha)$ is to select the action $\alpha$ with the highest expected reward $Q_t(\alpha)$ at time $t$:

$$\pi(\alpha) = \underset{\alpha}{argmax} \; Q_t(\alpha)$$

where the action space is given by the decision rule:

$$actions = \alpha = \{"shape", "colour", "number" \}$$

----- 
The simplist calculation of this would be to let the expected return of each action be the sample average return from trying that action. That is to say:

$$Q_t(\alpha) = \frac{1}{t-1}\sum_1^{t-1}r_i$$

where $r_t$ is the binary reward received after taking an action:

$$r_t \in\{0,1\}$$

Note: for faster computation & more efficient memory, this average calculation can be written recursively:

$$Q_t = Q_{t-1} + \frac{1}{n}[r_t - Q_{t-1}]$$

Two problems arise:
- This ignores non-stationarity (the rule changes) and as such more recent feedback must carry more weight.
- This ignores the interaction effect of the game, knowledge about one action actually gives great info about the other actions as they are mutually exclusive. Should we encode this? It can be learnt through more aggressive updating.

The $\frac{1}{n}$ term effectively weights all returns $r_i$ equally. Thus this issues can be mitigated by overweighting newer returns & underweighting older returns. This can be achieved by setting this updating parameter to a constant $\lambda$:

$$Q_t = Q_{t-1} + \lambda[r_t - Q_{t-1}]$$

-----

Recall that $r_t \in\{0,1\}$, thus:

If $r_t = 1$ (correct choice):

$$
\begin{equation}
\begin{split}
    Q_t &= Q_{t-1} + \lambda[r_t - Q_{t-1}] \\
    Q_t &= Q_{t-1} + \lambda[1 - Q_{t-1}] \\
    Q_t &= \lambda + (1-\lambda)Q_{t-1}
\end{split}
\end{equation}
$$

Or, if $r_t = 0$ (incorrect choice):

$$
\begin{equation}
\begin{split}
    Q_t &= Q_{t-1} + \lambda[r_t - Q_{t-1}] \\
    Q_t &= Q_{t-1} + \lambda[0 - Q_{t-1}] \\
    Q_t &= (1-\lambda)Q_{t-1} 
\end{split}
\end{equation}
$$

It's trivial to see that $\lambda=1$ (the extreme case) would be choice the desired response:


If $r_t = 1$ & $\lambda=1$:

$$
\begin{equation}
\begin{split}
    Q_t &= \lambda + (1-\lambda)Q_{t-1} \\
    Q_t &= 1 
\end{split}
\end{equation}
$$

Or, if $r_t = 0$ & $\lambda=1$:

$$
\begin{equation}
\begin{split}
    Q_t &= (1-\lambda)Q_{t-1} \\
    Q_t &= 0
\end{split}
\end{equation}
$$

Which is correct as only one choice can be the correct choice. This is the extreme weighting, where we only take the value of the most recent return.

-----

# RL Instance
On the implementation design:
- Most memory + compute constraints arise from the Bayesian model, as such we wish to store lists of the Q-values etc so that we can monitor the algorithm overtime.
- All $Q$ values are initialized at $0$.

In [168]:
wcst = WCST_Environment(all_items)


class RL_Instance:
    """Defining the RL model"""
    def __init__(self):
        self.actions  = ['colour', 'shape', 'number']
        self.Q = {'colour': [0],
                  'shape' : [0],
                  'number': [0]}
        self.a_t = [None]

    def forward(self, x):
        # --- pass through the model ---x
        Q_t = [self.Q['colour'][-1], self.Q['shape'][-1], self.Q['number'][-1]]
        at  = self.actions[np.argmin(Q_t)]
        self.a_t.append(at)
        return(at)
    


    # REQUIREMENTS
    # - Model (given trained parameters)
    # - Training Loop
    # - Trainable Parameters
    # - Graphics (training processes etc)





In [169]:
wcst = WCST_Environment(all_items)
x = wcst.sample_environment(2)
Q = {'colour': [10],
                  'shape' : [21],
                  'number': [3]}
actions  = ['colour', 'shape', 'number']
        

In [170]:
Q_t = [Q['colour'][-1], Q['shape'][-1], Q['number'][-1]]
actions[np.argmin(Q_t)]

'number'

In [171]:
rl = RL_Instance()

In [192]:
wcst = WCST_Environment(all_items)
x = wcst.sample_environment(2)
rl.forward(x)

'colour'